In [76]:
# look at tools/set_up_magics.ipynb
get_ipython().run_cell('# one_liner_str #SET_UP_MAGIC_BEGIN\n\nget_ipython().run_cell_magic(\'javascript\', \'\', \'// setup cpp code highlighting\\nIPython.CodeCell.options_default.highlight_modes["text/x-c++src"] = {\\\'reg\\\':[/^%%cpp/]} ;\')\n\n# creating magics\nfrom IPython.core.magic import register_cell_magic, register_line_magic\nfrom IPython.display import display, Markdown, HTML\nimport argparse\nfrom subprocess import Popen, PIPE\nimport random\nimport sys\nimport os\nimport shutil\nimport shlex\n\n@register_cell_magic\ndef save_file(args_str, cell, line_comment_start="#"):\n    parser = argparse.ArgumentParser()\n    parser.add_argument("fname")\n    parser.add_argument("--ejudge-style", action="store_true")\n    args = parser.parse_args(args_str.split())\n    \n    cell = cell if cell[-1] == \'\\n\' or args.no_eof_newline else cell + "\\n"\n    cmds = []\n    with open(args.fname, "w") as f:\n        f.write(line_comment_start + " %%cpp " + args_str + "\\n")\n        for line in cell.split("\\n"):\n            line_to_write = (line if not args.ejudge_style else line.rstrip()) + "\\n"\n            if line.startswith("%"):\n                run_prefix = "%run "\n                assert line.startswith(run_prefix)\n                cmds.append(line[len(run_prefix):].strip())\n                f.write(line_comment_start + " " + line_to_write)\n            else:\n                f.write(line_to_write)\n        f.write("" if not args.ejudge_style else line_comment_start + r" line without \\n")\n    for cmd in cmds:\n        display(Markdown("Run: `%s`" % cmd))\n        get_ipython().system(cmd)\n\n@register_cell_magic\ndef cpp(fname, cell):\n    save_file(fname, cell, "//")\n\n@register_cell_magic\ndef asm(fname, cell):\n    save_file(fname, cell, "//")\n    \n@register_cell_magic\ndef makefile(fname, cell):\n    assert not fname\n    save_file("makefile", cell.replace(" " * 4, "\\t"))\n        \n@register_line_magic\ndef p(line):\n    try:\n        expr, comment = line.split(" #")\n        display(Markdown("`{} = {}`  # {}".format(expr.strip(), eval(expr), comment.strip())))\n    except:\n        display(Markdown("{} = {}".format(line, eval(line))))\n        \ndef show_file(file, clear_at_begin=True, return_html_string=False):\n    if clear_at_begin:\n        get_ipython().system("truncate --size 0 " + file)\n    obj = file.replace(\'.\', \'_\').replace(\'/\', \'_\') + "_obj"\n    html_string = \'\'\'\n        <!--MD_BEGIN_FILTER-->\n        <script type=text/javascript>\n        var entrance___OBJ__ = 0;\n        var errors___OBJ__ = 0;\n        function refresh__OBJ__()\n        {\n            entrance___OBJ__ -= 1;\n            var elem = document.getElementById("__OBJ__");\n            if (elem) {\n                var xmlhttp=new XMLHttpRequest();\n                xmlhttp.onreadystatechange=function()\n                {\n                    var elem = document.getElementById("__OBJ__");\n                    console.log(!!elem, xmlhttp.readyState, xmlhttp.status, entrance___OBJ__);\n                    if (elem && xmlhttp.readyState==4) {\n                        if (xmlhttp.status==200)\n                        {\n                            errors___OBJ__ = 0;\n                            if (!entrance___OBJ__) {\n                                elem.innerText = xmlhttp.responseText;\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                            return xmlhttp.responseText;\n                        } else {\n                            errors___OBJ__ += 1;\n                            if (errors___OBJ__ < 10 && !entrance___OBJ__) {\n                                entrance___OBJ__ += 1;\n                                console.log("req");\n                                window.setTimeout("refresh__OBJ__()", 300); \n                            }\n                        }\n                    }\n                }\n                xmlhttp.open("GET", "__FILE__", true);\n                xmlhttp.setRequestHeader("Cache-Control", "no-cache");\n                xmlhttp.send();     \n            }\n        }\n        \n        if (!entrance___OBJ__) {\n            entrance___OBJ__ += 1;\n            refresh__OBJ__(); \n        }\n        </script>\n        \n        <font color="white"> <tt>\n        <p id="__OBJ__" style="font-size: 16px; border:3px #333333 solid; background: #333333; border-radius: 10px; padding: 10px;  "></p>\n        </tt> </font>\n        <!--MD_END_FILTER-->\n        <!--MD_FROM_FILE __FILE__ -->\n        \'\'\'.replace("__OBJ__", obj).replace("__FILE__", file)\n    if return_html_string:\n        return html_string\n    display(HTML(html_string))\n    \nBASH_POPEN_TMP_DIR = "./bash_popen_tmp"\n    \ndef bash_popen_terminate_all():\n    for p in globals().get("bash_popen_list", []):\n        print("Terminate pid=" + str(p.pid), file=sys.stderr)\n        p.terminate()\n    globals()["bash_popen_list"] = []\n    if os.path.exists(BASH_POPEN_TMP_DIR):\n        shutil.rmtree(BASH_POPEN_TMP_DIR)\n\nbash_popen_terminate_all()  \n\ndef bash_popen(cmd):\n    if not os.path.exists(BASH_POPEN_TMP_DIR):\n        os.mkdir(BASH_POPEN_TMP_DIR)\n    h = os.path.join(BASH_POPEN_TMP_DIR, str(random.randint(0, 1e18)))\n    stdout_file = h + ".out.html"\n    stderr_file = h + ".err.html"\n    run_log_file = h + ".fin.html"\n    \n    stdout = open(stdout_file, "wb")\n    stdout = open(stderr_file, "wb")\n    \n    html = """\n    <table width="100%">\n    <colgroup>\n       <col span="1" style="width: 70px;">\n       <col span="1">\n    </colgroup>    \n    <tbody>\n      <tr> <td><b>STDOUT</b></td> <td> {stdout} </td> </tr>\n      <tr> <td><b>STDERR</b></td> <td> {stderr} </td> </tr>\n      <tr> <td><b>RUN LOG</b></td> <td> {run_log} </td> </tr>\n    </tbody>\n    </table>\n    """.format(\n        stdout=show_file(stdout_file, return_html_string=True),\n        stderr=show_file(stderr_file, return_html_string=True),\n        run_log=show_file(run_log_file, return_html_string=True),\n    )\n    \n    cmd = """\n        bash -c {cmd} &\n        pid=$!\n        echo "Process started! pid=${{pid}}" > {run_log_file}\n        wait ${{pid}}\n        echo "Process finished! exit_code=$?" >> {run_log_file}\n    """.format(cmd=shlex.quote(cmd), run_log_file=run_log_file)\n    # print(cmd)\n    display(HTML(html))\n    \n    p = Popen(["bash", "-c", cmd], stdin=PIPE, stdout=stdout, stderr=stdout)\n    \n    bash_popen_list.append(p)\n    return p\n\n\n@register_line_magic\ndef bash_async(line):\n    bash_popen(line)\n    \ndef make_oneliner():\n    return \'# look at tools/set_up_magics.ipynb\\nget_ipython().run_cell(%s)\\nNone\' % repr(one_liner_str)\n\n #SET_UP_MAGIC_END \n')
None

<IPython.core.display.Javascript object>

# named FIFO

Ранее мы познакомились с пайпами (анонимными fifo (далее буду называть просто pipe'ами)). Теперь же посмотрим на именованые.
Отличие в том, что именоваванные fifo (дальше буду называть просто fifo) являются файлами в файловой системе linux. Соответственно они могут существовать, не будучи открытыми какой-либо программой. Как и файл их можно удалить.

Как создать из консоли - `man mkfifo`, как создать из кода на C - `man 3 mkfifo`. Чтение и запись в fifo происходит так же как и с обычным файлом.

**Важно:** fifo, это файл читаемый двумя процессами и важно, кто открыл процесс на запись, кто на чтение. Например, fifo не может быть открыта на запись, пока кто-нибудь не открыл ее на чтение.

In [62]:
!rm -f my_fifo
!mkfifo my_fifo

In [324]:
!echo "Hello" > my_fifo

^C
/bin/sh: 1: cannot create my_fifo: Interrupted system call


In [79]:
%bash_async echo "Hello" > my_fifo ; echo "After writing to my_fifo"

STDOUT,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/979360637155236027.out.html -->
STDERR,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/979360637155236027.err.html -->
RUN LOG,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/979360637155236027.fin.html -->


In [80]:
!cat my_fifo

Hello


# Теперь на С
Обратите внимание, что fifo не может открыться на запись, пока ее не начнут читать.

In [36]:
%%cpp write_fifo.cpp
%run gcc write_fifo.cpp -o write_fifo.exe

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <assert.h>
#include <sys/wait.h>
#include <fcntl.h>

int main() {
    fprintf(stderr, "Started\n"); fflush(stderr);
    int fd = open("my_fifo", O_WRONLY);
    assert(fd >= 0);
    fprintf(stderr, "Opened\n"); fflush(stderr);
    const char str[] = "Hello from C!\n";
    assert(write(fd, str, sizeof(str)) == sizeof(str));
    fprintf(stderr, "Written\n"); fflush(stderr);
    assert(close(fd) == 0);
    fprintf(stderr, "Closed\n"); fflush(stderr);
    return 0;
}

Run: `gcc write_fifo.cpp -o write_fifo.exe`

In [111]:
%bash_async ./write_fifo.exe

STDOUT,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/564617692879239134.out.html -->
STDERR,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/564617692879239134.err.html -->
RUN LOG,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/564617692879239134.fin.html -->


In [112]:
!cat my_fifo

Hello from C!
 

# Директория /proc/<pid\>/*

Интересная штука директория `/proc` это виртулаьная файловая система в которой можно получать сведения о процессах, читая из из файлов. (Это не обычные файлы на диске, а скорее некоторое view на сведения о процессах из ядра системы).

Что есть в proc: http://man7.org/linux/man-pages/man5/proc.5.html

Имеющая отношение к делу статья на хабре: https://habr.com/ru/post/209446/

Посмотрим, что можно узнать о запущенном процессе:

In [113]:
# запустим процесс в фоне
%bash_async echo "Hello" > my_fifo 

STDOUT,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/452907451508078450.out.html -->
STDERR,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/452907451508078450.err.html -->
RUN LOG,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/452907451508078450.fin.html -->


In [115]:
!cat /proc/9697/status

Name:	bash
Umask:	0002
State:	S (sleeping)
Tgid:	9697
Ngid:	0
Pid:	9697
PPid:	9696
TracerPid:	0
Uid:	1000	1000	1000	1000
Gid:	1000	1000	1000	1000
FDSize:	64
Groups:	4 24 27 30 46 113 128 130 999 1000 
NStgid:	9697
NSpid:	9697
NSpgid:	4079
NSsid:	4079
VmPeak:	   19596 kB
VmSize:	   19596 kB
VmLck:	       0 kB
VmPin:	       0 kB
VmHWM:	    3148 kB
VmRSS:	    3148 kB
RssAnon:	     196 kB
RssFile:	    2952 kB
RssShmem:	       0 kB
VmData:	     176 kB
VmStk:	     132 kB
VmExe:	     976 kB
VmLib:	    2112 kB
VmPTE:	      72 kB
VmSwap:	       0 kB
HugetlbPages:	       0 kB
CoreDumping:	0
Threads:	1
SigQ:	4/7735
SigPnd:	0000000000000000
ShdPnd:	0000000000000000
SigBlk:	0000000000000000
SigIgn:	0000000000000006
SigCgt:	0000000000010000
CapInh:	0000000000000000
CapPrm:	0000000000000000
CapEff:	0000000000000000
CapBnd:	0000003fffffffff
CapAmb:	0000000000000000
NoNewPrivs:	0
Seccomp:	0
Speculation_Store_Bypass:	vulnerable
Cpus_allowed:	1
Cpus_allowed_list:	0
Mems_allowed:	00000000,00000000,0000000

In [116]:
!cat my_fifo

Hello


In [117]:
!ps aux | grep write_fifo 

pechatn+  9704  0.0  0.0   4504   752 pts/20   Ss+  01:05   0:00 /bin/sh -c ps aux | grep write_fifo 
pechatn+  9706  0.0  0.0  21292  1084 pts/20   S+   01:05   0:00 grep write_fifo


# Пример применения на моей практике
Только на семинаре

# Пример, почему важно правильно открывать fifo

In [118]:
import os

In [119]:
fd = os.open("my_fifo", os.O_RDWR) # создаем ненужное открытие файла на запись

In [120]:
%bash_async cat my_fifo

STDOUT,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/11080873394057405.out.html -->
STDERR,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/11080873394057405.err.html -->
RUN LOG,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/11080873394057405.fin.html -->


In [121]:
!echo "Hello 1" > my_fifo
!echo "Hello 2" > my_fifo
!echo "Hello 3" > my_fifo

In [122]:
os.close(fd) # Только после закрытия дескриптора процесс 'cat my_fifo' завершится. Так как закроется fifo

### Если же ненужного чтения не создавать:

In [123]:
%bash_async cat my_fifo

STDOUT,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/859234153764631452.out.html -->
STDERR,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/859234153764631452.err.html -->
RUN LOG,<!--MD_BEGIN_FILTER--> <!--MD_END_FILTER--> <!--MD_FROM_FILE ./bash_popen_tmp/859234153764631452.fin.html -->


In [124]:
!echo "Hello 1" > my_fifo

In [125]:
!echo "Hello 2" > my_fifo # то все зависнет тут

^C
/bin/sh: 1: cannot create my_fifo: Interrupted system call
